# Preston's Test file :)

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Loading feature data

In [90]:
data_df_full = pd.read_csv('datasets/TRY_Categorical_Traits_Lookup_Table_2012_03_17_TestRelease.csv')
print('size:', data_df_full.size)
data_df_full.head()

size: 3104021


C:\Users\whitcombp\AppData\Local\Temp\ipykernel_23752\767167101.py:1: DtypeWarning: Columns (8,10,12,13,19,21,23,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df_full = pd.read_csv('datasets/TRY_Categorical_Traits_Lookup_Table_2012_03_17_TestRelease.csv')


,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,80,Abronia maritima,x,Abronia,maritima,Nyctaginaceae,Angiosperm_Eudicotyl,herb,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,139,Abutilon theophrasti,x,Abutilon,theophrasti,Malvaceae,Angiosperm_Eudicotyl,herb,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169,Acacia aneura,x,Acacia,aneura,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,197,Acacia binervata,x,Acacia,binervata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,218,Acacia burkittii,x,Acacia,burkittii,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
source_columns = data_df_full.columns.str.contains('source', case=False)
first_col = np.where(source_columns)[0][0]
last_col = len(data_df_full.columns)
df = data_df_full.drop(data_df_full.columns[list(range(first_col, last_col))], axis=1)

In [138]:
df[df['AccSpeciesName'].str.contains('magnificum', case=False)].head(10)

,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,Epiphyte,Crop,Palmoid,LeafType,LeafPhenology,PhotosyntheticPathway,Woodiness,WoodinessDetail,LeafCompoundness,NumberOfLeaflets
50407,65294,Adenocalymma magnificum,x,Adenocalymma,magnificum,Bignoniaceae,Angiosperm_Eudicotyl,NaN,NaN,NaN,...,NaN,NaN,NaN,broadleaved,NaN,NaN,NaN,NaN,NaN,NaN


## loading domes' plants data

In [155]:
domes_plants_df_full = pd.read_csv("Plants_Formatted.csv", encoding="latin-1")
domes_plants_df_full.head()

,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,Silver-Dollar Fern,Adiantum peruvianum,3,\r\n,x,x,\r\n,\r\n
1,Maidenhair Fern,Adiantum raddianum,2,\r\n,x,x,\r\n,\r\n
2,Rosy Maidenhair Fern,Adiantum ternerum 'Scutum Roseum',3,\r\n,x,x,\r\n,\r\n
3,Diamond Maidenhair Fern,Adiantum trapeziforme,6,\r\n,x,x,\r\n,\r\n
4,Living Vase Bromeliad,Aechmea 'Better than Bert',2,\r\n,x,x,\r\n,"one is potted, easily transplanted"


In [167]:
def clean(string : str) -> str:
    # full string cleaning
    string = string.lower()

    # string formatting cleaning
    substrings = string.split(' ')
    ignore = ['\'' in s for s in substrings]
    ignore.append(True) # ensures it exists in array since index() fails otherwise
    nonignored = substrings[:ignore.index(True)] # we don't care about anything after the first '
    if len(nonignored) <= 1: # can't skip first word, too short
        return nonignored[0] # remove from list for consistency
    else:
        nonignored = nonignored[1:] # skip first word (genus)
        return ' '.join(nonignored)

In [168]:
clean("Anthurium 'Big Red Bird' x A. 'Round Wan'")

'anthurium'

In [163]:
domes_plants_df_full['Shortened Scientific'] = domes_plants_df_full['Scientific Name'].apply(clean)
domes_plants_df_full['Shortened Scientific'].head()

0      peruvianum
1       raddianum
2        ternerum
3    trapeziforme
4         Aechmea
Name: Shortened Scientific, dtype: object

In [169]:
df['Shortened Scientific'] = df['AccSpeciesName'].apply(clean)
df['Shortened Scientific'].head()

0       maritima
1    theophrasti
2         aneura
3      binervata
4      burkittii
Name: Shortened Scientific, dtype: object

## joining

In [170]:
joined = df.merge(domes_plants_df_full, on='Shortened Scientific')

In [172]:
print('size =', joined.shape)
joined.head()

size = (3817, 31)


,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,NumberOfLeaflets,Shortened Scientific,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,302,Acacia elata,x,Acacia,elata,Fabaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,elata,SOAP TREE,Yucca elata,2,\r\n,\r\n,X,\r\n,\r\n
1,1390,Aegilops comosa,x,Aegilops,comosa,Poaceae,Angiosperm_Monocotyl,graminoid,NaN,NaN,...,NaN,comosa,Pineapple Lily,Eucomis comosa,1,\r\n,x,x,\r\n,potted
2,1595,Aextoxicon punctatum,x,Aextoxicon,punctatum,Aextoxicaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,punctatum,Elkhorn Fern,Microsorum punctatum 'Serratum',1,\r\n,x,x,\r\n,growing on a rock
3,2221,Albizia chinensis,x,Albizia,chinensis,Fabaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,chinensis,Jojoba,Simmondsia chinensis,3,X,\r\n,X,\r\n,Largest plant unlikely to survive but smaller ...
4,4257,Apoplanesia paniculata,x,Apoplanesia,paniculata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,paniculata,Orange Jasmine,Murraya paniculata,2,\r\n,x,x,\r\n,\r\n


## More cleaning!! :)

In [174]:
joined = joined.dropna(how='all')
joined = joined.fillna(0)
joined.head()

,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,NumberOfLeaflets,Shortened Scientific,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,302,Acacia elata,x,Acacia,elata,Fabaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,elata,SOAP TREE,Yucca elata,2,\r\n,\r\n,X,\r\n,\r\n
1,1390,Aegilops comosa,x,Aegilops,comosa,Poaceae,Angiosperm_Monocotyl,graminoid,0,0,...,0,comosa,Pineapple Lily,Eucomis comosa,1,\r\n,x,x,\r\n,potted
2,1595,Aextoxicon punctatum,x,Aextoxicon,punctatum,Aextoxicaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,punctatum,Elkhorn Fern,Microsorum punctatum 'Serratum',1,\r\n,x,x,\r\n,growing on a rock
3,2221,Albizia chinensis,x,Albizia,chinensis,Fabaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,chinensis,Jojoba,Simmondsia chinensis,3,X,\r\n,X,\r\n,Largest plant unlikely to survive but smaller ...
4,4257,Apoplanesia paniculata,x,Apoplanesia,paniculata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,0,0,...,0,paniculata,Orange Jasmine,Murraya paniculata,2,\r\n,x,x,\r\n,\r\n


# Testing New Idea!!!!
Trying to use kaggle dataset with existing pair labels to extrapolate the data to larger plant species dataset  
Process:  
 - take labeled kaggle dataset
 - expand dataset to TRY plant features
 - fit supervised learning model
 - test with larger TRY plant dataset

In [177]:
pairs_df_full = pd.read_csv('datasets/companion_plants.csv')
print('shape:', pairs_df_full.shape)
pairs_df_full.head(10)

shape: (995, 4)


,Source Node,Link,Destination Node,Source Type
0,alliums,helps,fruit trees,vegetables
1,alliums,helps,nightshades,vegetables
2,alliums,helps,tomatoes,vegetables
3,alliums,helps,capsicum,vegetables
4,alliums,helps,peppers,vegetables
5,alliums,helps,potatoes,vegetables
6,alliums,helps,brassicas,vegetables
7,alliums,helps,carrots,vegetables
8,asparagus,helps,tomatoes,vegetables
9,asparagus,helps,parsley,vegetables


In [180]:
from pygbif import occurrences

# Example: Get 300 occurrences of "Solanum lycopersicum" (tomato)
res = occurrences.search(
    scientificName="Aegilops comosa",
    hasCoordinate=True,
    limit=300
)

# Convert to a DataFrame
import pandas as pd
df = pd.DataFrame(res["results"])

# Keep essential geographic fields
df = df[["species", "decimalLatitude", "decimalLongitude", 
         "eventDate", "basisOfRecord", "coordinateUncertaintyInMeters"]]

print(df.head())


           species  decimalLatitude  decimalLongitude   eventDate  \
0  Aegilops comosa         37.33833          22.65500  2018-05-20   
1  Aegilops comosa         37.52389          22.63528  2018-05-18   
2  Aegilops comosa         37.32889          22.58000  2018-05-24   
3  Aegilops comosa         37.61667          22.60639  2018-05-17   
4  Aegilops comosa         37.51500          23.10167  2018-05-14   

        basisOfRecord  coordinateUncertaintyInMeters  
0  PRESERVED_SPECIMEN                            NaN  
1  PRESERVED_SPECIMEN                            NaN  
2  PRESERVED_SPECIMEN                            NaN  
3  PRESERVED_SPECIMEN                            NaN  
4  PRESERVED_SPECIMEN                            NaN  
